In [1]:
import requests

response = requests.get('https://www.tally.xyz/api/search-daos')

import pandas as pd
df = pd.DataFrame(response.json()['governances'])
df.head()

,id,slug,name,kind,chainId,organization
0,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,arbitrum-dao-grants-by-qb,Arbitrum DAO Grants by QB,SINGLE_GOV,eip155:42161,"{'name': 'Arbitrum DAO Grants by QB', 'slug': ..."
1,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,bound-finance-dao,Bound Finance DAO,SINGLE_GOV,eip155:1,"{'name': 'Bound Finance DAO', 'slug': 'bound-f..."
2,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,tai,TAI Governance,SINGLE_GOV,eip155:1,"{'name': 'TAI Governance', 'slug': 'tai-govern..."
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,sector-3-optimism,Sector#3 (Optimism),SINGLE_GOV,eip155:10,"{'name': 'Sector#3 (Optimism)', 'slug': 'secto..."
4,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,nexus,CoinDaoX,SINGLE_GOV,eip155:137,"{'name': 'CoinDaoX', 'slug': 'coindaox', 'visu..."


In [4]:
', '.join(df.chainId.unique())

'eip155:42161, eip155:1, eip155:10, eip155:137, eip155:100, eip155:84531, eip155:1284, eip155:56, eip155:43114'

In [2]:
df.columns

Index(['id', 'slug', 'name', 'kind', 'chainId', 'organization'], dtype='object')

In [3]:
# expand the organization column
df = pd.concat([df.drop(['organization'], axis=1), df['organization'].apply(pd.Series)], axis=1)
df.head()

,id,slug,name,kind,chainId,name,slug,visual,governances
0,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,arbitrum-dao-grants-by-qb,Arbitrum DAO Grants by QB,SINGLE_GOV,eip155:42161,Arbitrum DAO Grants by QB,arbitrum-dao-grants-by-qb,{'icon': ''},"[{'stats': {'proposals': {'active': 0, 'total'..."
1,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,bound-finance-dao,Bound Finance DAO,SINGLE_GOV,eip155:1,Bound Finance DAO,bound-finance-dao,{'icon': ''},"[{'stats': {'proposals': {'active': 0, 'total'..."
2,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,tai,TAI Governance,SINGLE_GOV,eip155:1,TAI Governance,tai-governance-2,{'icon': 'https://static.tally.xyz/3cda4f61-fa...,"[{'stats': {'proposals': {'active': 0, 'total'..."
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,sector-3-optimism,Sector#3 (Optimism),SINGLE_GOV,eip155:10,Sector#3 (Optimism),sector-3-optimism,{'icon': 'https://static.tally.xyz/268e141b-ec...,"[{'stats': {'proposals': {'active': 0, 'total'..."
4,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,coindaox,CoinDaoX,SINGLE_GOV,eip155:137,CoinDaoX,coindaox,{'icon': 'https://static.tally.xyz/24c28289-66...,"[{'stats': {'proposals': {'active': 0, 'total'..."


In [4]:
df.iloc[0]['governances']

[{'stats': {'proposals': {'active': 0, 'total': 0},
   'tokens': {'owners': 613174, 'voters': 131995}}}]

In [5]:
df['proposals_count'] = df['governances'].apply(lambda x: sum([g['stats']['proposals']['total'] for g in x]))
df['votes_count'] = df['governances'].apply(lambda x: sum([g['stats']['tokens']['voters'] for g in x]))
df

,id,slug,name,kind,chainId,name,slug,visual,governances,proposals_count,votes_count
0,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,arbitrum-dao-grants-by-qb,Arbitrum DAO Grants by QB,SINGLE_GOV,eip155:42161,Arbitrum DAO Grants by QB,arbitrum-dao-grants-by-qb,{'icon': ''},"[{'stats': {'proposals': {'active': 0, 'total'...",0,131995
1,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,bound-finance-dao,Bound Finance DAO,SINGLE_GOV,eip155:1,Bound Finance DAO,bound-finance-dao,{'icon': ''},"[{'stats': {'proposals': {'active': 0, 'total'...",2,1
2,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,tai,TAI Governance,SINGLE_GOV,eip155:1,TAI Governance,tai-governance-2,{'icon': 'https://static.tally.xyz/3cda4f61-fa...,"[{'stats': {'proposals': {'active': 0, 'total'...",0,2
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,sector-3-optimism,Sector#3 (Optimism),SINGLE_GOV,eip155:10,Sector#3 (Optimism),sector-3-optimism,{'icon': 'https://static.tally.xyz/268e141b-ec...,"[{'stats': {'proposals': {'active': 0, 'total'...",1,4
4,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,coindaox,CoinDaoX,SINGLE_GOV,eip155:137,CoinDaoX,coindaox,{'icon': 'https://static.tally.xyz/24c28289-66...,"[{'stats': {'proposals': {'active': 0, 'total'...",1,1
...,...,...,...,...,...,...,...,...,...,...,...
575,eip155:1:0x0BEF27FEB58e857046d630B2c03dFb7bae5...,fei,Fei,SINGLE_GOV,eip155:1,Fei,fei,{'icon': 'https://static.withtally.com/c180925...,"[{'stats': {'proposals': {'active': 0, 'total'...",64,335
576,eip155:137:0x9B3a9e3f289095A7E1117deb3bD29Fd7C...,under3,Under3,SINGLE_GOV,eip155:137,Under3,under3,{'icon': 'https://static.withtally.com/a5054e9...,"[{'stats': {'proposals': {'active': 0, 'total'...",7,2
577,eip155:137:0x89de859404FeAC5346f5a843Db6B5E136...,harry-potter-82,Harry Potter,SINGLE_GOV,eip155:137,Harry Potter,harry-potter-82,{'icon': 'https://static.withtally.com/131bcd2...,"[{'stats': {'proposals': {'active': 0, 'total'...",6,1
578,eip155:137:0x806713eAAAAFEBfb4F0f6d6ED54775f76...,allturn-space-83,AllTurn.Space,SINGLE_GOV,eip155:137,AllTurn.Space,allturn.space,{'icon': 'https://static.withtally.com/5dcfedc...,"[{'stats': {'proposals': {'active': 0, 'total'...",3,1


In [6]:
df = df[['id', 'slug', 'name', 'votes_count', 'proposals_count']]
# drop one of the slug columns
df.head()

,id,slug,slug,name,name,votes_count,proposals_count
0,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,arbitrum-dao-grants-by-qb,arbitrum-dao-grants-by-qb,Arbitrum DAO Grants by QB,Arbitrum DAO Grants by QB,131995,0
1,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,bound-finance-dao,bound-finance-dao,Bound Finance DAO,Bound Finance DAO,1,2
2,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,tai,tai-governance-2,TAI Governance,TAI Governance,2,0
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,sector-3-optimism,sector-3-optimism,Sector#3 (Optimism),Sector#3 (Optimism),4,1
4,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,coindaox,coindaox,CoinDaoX,CoinDaoX,1,1


In [7]:
# add platform column
df['platform'] = 'tally'
df = df.rename(columns={'id': 'platform_id'})
df

/var/folders/yn/b6240_995k3gl4vwtnbzzy480000gq/T/ipykernel_21457/301900197.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['platform'] = 'tally'


,platform_id,slug,slug,name,name,votes_count,proposals_count,platform
0,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,arbitrum-dao-grants-by-qb,arbitrum-dao-grants-by-qb,Arbitrum DAO Grants by QB,Arbitrum DAO Grants by QB,131995,0,tally
1,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,bound-finance-dao,bound-finance-dao,Bound Finance DAO,Bound Finance DAO,1,2,tally
2,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,tai,tai-governance-2,TAI Governance,TAI Governance,2,0,tally
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,sector-3-optimism,sector-3-optimism,Sector#3 (Optimism),Sector#3 (Optimism),4,1,tally
4,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,coindaox,coindaox,CoinDaoX,CoinDaoX,1,1,tally
...,...,...,...,...,...,...,...,...
575,eip155:1:0x0BEF27FEB58e857046d630B2c03dFb7bae5...,fei,fei,Fei,Fei,335,64,tally
576,eip155:137:0x9B3a9e3f289095A7E1117deb3bD29Fd7C...,under3,under3,Under3,Under3,2,7,tally
577,eip155:137:0x89de859404FeAC5346f5a843Db6B5E136...,harry-potter-82,harry-potter-82,Harry Potter,Harry Potter,1,6,tally
578,eip155:137:0x806713eAAAAFEBfb4F0f6d6ED54775f76...,allturn-space-83,allturn.space,AllTurn.Space,AllTurn.Space,1,3,tally


In [8]:
# make website column from https://www.tally.xyz/gov/<platform_id>
df['website'] = 'https://www.tally.xyz/gov/' + df['platform_id']
df

,platform_id,slug,slug,name,name,votes_count,proposals_count,platform,website
0,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,arbitrum-dao-grants-by-qb,arbitrum-dao-grants-by-qb,Arbitrum DAO Grants by QB,Arbitrum DAO Grants by QB,131995,0,tally,https://www.tally.xyz/gov/eip155:42161:0x32E19...
1,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,bound-finance-dao,bound-finance-dao,Bound Finance DAO,Bound Finance DAO,1,2,tally,https://www.tally.xyz/gov/eip155:1:0x28Bc951D5...
2,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,tai,tai-governance-2,TAI Governance,TAI Governance,2,0,tally,https://www.tally.xyz/gov/eip155:1:0x90F2273a7...
3,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,sector-3-optimism,sector-3-optimism,Sector#3 (Optimism),Sector#3 (Optimism),4,1,tally,https://www.tally.xyz/gov/eip155:10:0x8087dE46...
4,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,coindaox,coindaox,CoinDaoX,CoinDaoX,1,1,tally,https://www.tally.xyz/gov/eip155:137:0x3343aEb...
...,...,...,...,...,...,...,...,...,...
575,eip155:1:0x0BEF27FEB58e857046d630B2c03dFb7bae5...,fei,fei,Fei,Fei,335,64,tally,https://www.tally.xyz/gov/eip155:1:0x0BEF27FEB...
576,eip155:137:0x9B3a9e3f289095A7E1117deb3bD29Fd7C...,under3,under3,Under3,Under3,2,7,tally,https://www.tally.xyz/gov/eip155:137:0x9B3a9e3...
577,eip155:137:0x89de859404FeAC5346f5a843Db6B5E136...,harry-potter-82,harry-potter-82,Harry Potter,Harry Potter,1,6,tally,https://www.tally.xyz/gov/eip155:137:0x89de859...
578,eip155:137:0x806713eAAAAFEBfb4F0f6d6ED54775f76...,allturn-space-83,allturn.space,AllTurn.Space,AllTurn.Space,1,3,tally,https://www.tally.xyz/gov/eip155:137:0x806713e...


In [9]:
# add an empty additional column
df['additional'] = None
df = df[['platform', 'platform_id', 'name', 'additional', 'website', 'votes_count', 'proposals_count']]

In [10]:
df.head()

,platform,platform_id,name,name,additional,website,votes_count,proposals_count
0,tally,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,Arbitrum DAO Grants by QB,Arbitrum DAO Grants by QB,None,https://www.tally.xyz/gov/eip155:42161:0x32E19...,131995,0
1,tally,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,Bound Finance DAO,Bound Finance DAO,None,https://www.tally.xyz/gov/eip155:1:0x28Bc951D5...,1,2
2,tally,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,TAI Governance,TAI Governance,None,https://www.tally.xyz/gov/eip155:1:0x90F2273a7...,2,0
3,tally,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,Sector#3 (Optimism),Sector#3 (Optimism),None,https://www.tally.xyz/gov/eip155:10:0x8087dE46...,4,1
4,tally,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,CoinDaoX,CoinDaoX,None,https://www.tally.xyz/gov/eip155:137:0x3343aEb...,1,1


In [11]:
# fix dupe name col
df.columns = ['platform', 'platform_id', 'name', 'name_duplicate', 'additional', 'website', 'votes_count', 'proposals_count']
df.head()

,platform,platform_id,name,name_duplicate,additional,website,votes_count,proposals_count
0,tally,eip155:42161:0x32E197aAD52d672c1b0fC15460645bc...,Arbitrum DAO Grants by QB,Arbitrum DAO Grants by QB,None,https://www.tally.xyz/gov/eip155:42161:0x32E19...,131995,0
1,tally,eip155:1:0x28Bc951D50A60Fe073eDD4dB4CC9330dd5B...,Bound Finance DAO,Bound Finance DAO,None,https://www.tally.xyz/gov/eip155:1:0x28Bc951D5...,1,2
2,tally,eip155:1:0x90F2273a793B2E883196fC0dB72a2da483e...,TAI Governance,TAI Governance,None,https://www.tally.xyz/gov/eip155:1:0x90F2273a7...,2,0
3,tally,eip155:10:0x8087dE46076212A21EA5cD8b4368BCbd82...,Sector#3 (Optimism),Sector#3 (Optimism),None,https://www.tally.xyz/gov/eip155:10:0x8087dE46...,4,1
4,tally,eip155:137:0x3343aEb44A2a277e3Ae346014e181aE15...,CoinDaoX,CoinDaoX,None,https://www.tally.xyz/gov/eip155:137:0x3343aEb...,1,1


In [12]:
# drop dupe name col
df = df.drop(['name_duplicate'], axis=1)

In [13]:
# save
df.to_csv('tally_deployments.csv', index=False)